In [ ]:
# | default_exp _helpers.utils

# Utils

In [ ]:
# | export

import random
import time

import openai
import typer

In [ ]:
import pytest

In [ ]:
# | export

# Reference: https://github.com/openai/openai-cookbook/blob/main/examples/How_to_handle_rate_limits.ipynb

# define a retry decorator
def retry_with_exponential_backoff(
    initial_delay: float = 1,
    exponential_base: float = 2,
    jitter: bool = True,
    max_retries: int = 10,
    max_wait: float = 60,
    errors: tuple = (openai.error.RateLimitError, openai.error.ServiceUnavailableError),
):
    """Retry a function with exponential backoff."""

    def decorator(func):
        def wrapper(*args, **kwargs):
            num_retries = 0
            delay = initial_delay

            while True:
                try:
                    return func(*args, **kwargs)

                except errors as e:
                    num_retries += 1
                    if num_retries > max_retries:
                        raise Exception(
                            f"Maximum number of retries ({max_retries}) exceeded."
                        )
                    delay = min(
                        delay * exponential_base * (1 + jitter * random.random()), # nosec
                        max_wait,
                    )
                    typer.secho(
                        f"Note: OpenAI's API rate limit reached. Command will automatically retry in {int(delay)} seconds. For more information visit: https://help.openai.com/en/articles/5955598-is-api-usage-subject-to-any-rate-limits",
                        fg=typer.colors.BLUE,
                    )
                    time.sleep(delay)

                except Exception as e:
                    raise e

        return wrapper

    return decorator


@retry_with_exponential_backoff()
def completions_with_backoff(**kwargs):
    """


    !!! note
    
        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    
    
    
    
    
    return openai.Completion.create(**kwargs)

In [ ]:
@retry_with_exponential_backoff()
def mock_func():
    """


    !!! note
    
        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    
    
    
    
    
    return "Success"

assert mock_func() == "Success"

# Test max retries exceeded
@retry_with_exponential_backoff(max_retries=1)
def mock_func_error():
    """


    !!! note
    
        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    
    
    
    
    
    raise openai.error.RateLimitError

with pytest.raises(Exception) as e:
    mock_func_error()
    
print(e.value)
assert str(e.value) == "Maximum number of retries (1) exceeded."

Note: OpenAI's API rate limit reached. Command will automatically retry in 2 seconds. For more information visit: https://help.openai.com/en/articles/5955598-is-api-usage-subject-to-any-rate-limits
Maximum number of retries (1) exceeded.
